# Fase 0: <i>Recopilación de datos</i>
## <b>Obtención de los vuelos y precios de <i>vuelosbaratos.es</i></b>

&emsp;&emsp;Con este archivo se procede a recopilar la información de los vuelos y precios de la página <b>vuelosbaratos.es</b>.

In [1]:
import pandas as pd
from time import *
from datetime import *
import requests
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
from random import randint
import html
from os import *
import locale
locale.setlocale(locale.LC_TIME, '')
rutas = "../../data/rutasIniciales.csv"
vuelos_data = "../../data/Datos/data_vuelosbaratos.csv"

In [2]:
listaRutas[0]
indice = 0
for i in range(len(listaRutas)):
    if listaRutas[i][0] == 'MAD':
        indice +=1
print (indice)

NameError: name 'listaRutas' is not defined

In [ ]:
listaRutas[0][0]

&emsp;&emsp;En primer lugar, obtenemos una lista con todas las rutas únicas (una ruta puede aparecer más de una vez al ser de diferentes compañías aéreas)

In [3]:
dfRutas = pd.read_csv(rutas)
dfRutas = dfRutas.sort_values(["Source Airport", "Destination Airport"])
origenes = dfRutas["Source Airport"].values
destinos = dfRutas["Destination Airport"].values
setRutas = set()
print("longitud origenes:", len(origenes))
for x,y in zip(origenes,destinos):
    ruta = x,y
    setRutas.add(ruta)
listaRutas = list(setRutas)
print("longitud setRutas"), len(setRutas)

longitud origenes: 721
longitud setRutas


(None, 321)

<table>
    <tr><td><b>Función:</b></td><td><i>getDetalle</i></td></tr>
    <tr><td><b>Entrada:</b></td><td><i>vuelo</i> Datos del vuelo recopilados en la página web</td></tr>
    <tr>
        <td><b>Salida:</b></td>
        <td style=" text-align:left;">
            <table style="width:100%">
                <tr>
                    <td style="width:15%"><b>salida_ida</b></td>
                    <td>Fecha y hora de salida del vuelo de ida</td>
                </tr>
                <tr>
                    <td><b>llegada_ida</b></td>
                    <td>Fecha y hora de llegada del vuelo de ida</td>
                </tr>
                <tr>
                    <td><b>aerolinea</b></td>
                    <td>Compañía aérea que realiza el vuelo</td>
                </tr>
                <tr>
                    <td><b>salida_vuelta</b></td>
                    <td>Fecha y hora de salida del vuelo de vuelta</td>
                </tr>
                <tr>
                    <td><b>llegada_vuelta</b></td>
                    <td>Fecha y hora de llegada del vuelo de vuelta</td>
                </tr>
            </table>
        </td>
    </tr>
    <tr>
        <td><b>Descripción:</b></td>
        <td>Recibe el texto HTML de un vuelo y lo procesa para devolver la fecha y hora de los vuelos de ida y de vuelta así como la compañía aérea que realiza el vuelo</td>
    </tr>

In [ ]:
def getDetalle(vuelo):
    salida_ida = ""
    llegada_ida = ""
    salida_vuelta = ""
    llegada_vuelta = ""
    aerolinea="" 
    aux = vuelo.find('table', class_='tblDetails')
    listaFilas = aux.find_all('tr')
    #En la primer fila de la tabla aparece la hora de salida y la hora de llegada del vuelo de ida y la aerolínea
    listaColumnas = listaFilas[0].find_all('td')
    salida_ida = datetime.strptime(listaColumnas[1].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
    llegada_ida = datetime.strptime(listaColumnas[2].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
    aerolinea = listaColumnas[4].find('img', class_= 'tail1')['alt']
    
    #En la segunda fila de la tabla aparece la hora de salida y la hora de llegada del vuelo de vuelta
    listaColumnas = listaFilas[1].find_all('td')
    salida_vuelta = datetime.strptime(listaColumnas[1].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
    llegada_vuelta = datetime.strptime(listaColumnas[2].find('span', class_='groupDate').text, '%d %b %Y, %H:%M').strftime('%Y-%m-%d %H:%M')
    
    return salida_ida, llegada_ida, aerolinea, salida_vuelta, llegada_vuelta

<table>
    <tr><td><b>Función:</b></td><td>procesaEnlace</td></tr>
    <tr>
        <td><b>Entrada:</b></td>
        <td>
            <table style='width:100%'>
                <tr>
                    <td style="width:15%"><b>empresa:</b></td>
                    <td> Nombre de la empresa que comercializa el billete</td>
                </tr>
                <tr>
                    <td><b>enlace</b></td>
                    <td>Enlace completo</td>
                </tr>
            </table>
        </td>
    </tr>
    <tr>
        <td><b>Salida:</b></td>
        <td>URL procesada</td>
    </tr>
    <tr>
        <td><b>Descripción:</b></td>
        <td>Reemplaza la codificación de la URL real en donde se vende realmente el billete para aquellas URLs que funcionan (en función de la empresa que lo venda) y devuelve el enlace listo para ser clickado</td>
    </tr>
</table>

In [ ]:
def procesaEnlace(empresa,enlace):
    respuesta = ""
    url = ""
    if empresa not in ["AirFrance","KLM","Ryanair"]:
        pos= enlace.find("url")
        respuesta = enlace[pos+4:]
        respuesta = respuesta.replace ('%21','!').replace ('%23','#').replace ('%24','$').replace ('%26','&').replace ('%27',"'").replace ('%28','(').replace ('%29',')').replace ('%2A','*').replace ('%2B', '+').replace ('%2C', ',').replace ('%2F','/').replace ('%3A',':').replace ('%3B',';').replace ('%3D','=').replace ('%3F','?').replace ('%40','@').replace ('%5B','[').replace ('%5D',']')
    return respuesta

&emsp;&emsp;Datos de búsqueda de vuelos (fecha de inicio de la búsqueda, número de rutas que se deben procesar para informar del progreso al usuario y número de días que se van a procesar)

In [ ]:
incrementoDia = timedelta(days=1)
incrementoVuelta = timedelta(days=7)
fecha = datetime(2020,12,29)
fechaVuelta = fecha + incrementoVuelta
totalRutas = 321
diasProcesar = 2

&emsp;&emsp;En la siguiente celda se procede a realizar la ejecución completa del webscraping, formateando las fechas para crear correctamente la URL a consultar, abriendo el navegador en modo incógnito y controlando los posibles errores que se pudieran producir para evitar que se interrumpa la obtención de los datos por ralentización en el flujo de la aplicación o por la aparición de cookies

In [ ]:
%%time

textoFecha = fecha.strftime("%Y-%#m-%#d")
textoFechaVuelta = fechaVuelta.strftime("%Y-%#m-%#d")
chrome_driver = "chromedriver.exe"

#Opción para abrir Chrome en modo incógnito
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
navegador = webdriver.Chrome(executable_path=chrome_driver,options=chrome_options)

numErrores = 0
for dia in range(diasProcesar): #Obtiene los datos para cada día
    dfVuelos = pd.DataFrame(columns=['CAPTURA','FUENTE','ORIGEN', 'DESTINO', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA', 'PRECIO', "URL", "AEROLINEA"])
    numeroRuta = 0
    #Recorre todas las rutas encontradas
    for ruta in listaRutas:
        error = True
        numeroRuta += 1
        origen = ruta[0]
        destino = ruta[1]
        print(datetime.now().strftime("%H:%M:%S")," ==> Procesando ruta nº " + str(numeroRuta) + " de " + str(totalRutas) + " RUTA:", origen,"-",destino, "Para el día", textoFecha)
        url = "https://www.vuelosbaratos.es/Buscar/" + origen + "-" + destino + "/" + textoFecha + '/' + textoFechaVuelta + "/ES/"
        navegador.implicitly_wait(20)
        while error:
            try:
                navegador.get(url)
                sleep(randint(4,8))
                source = navegador.page_source
                soup = BeautifulSoup(source).body
                #Obtengo la paginación y la limitamos a 5 páginas de vuelos más baratos
                numPaginas = 5
                numPag = len(soup.find_all('a', class_='ChangePage'))
                if numPag == 0:
                    error = False
                elif numPag < numPaginas:
                    numPaginas = numPag
                for numero in range(numPaginas):
                    #Obtengo la información de los vuelos
                    listaVuelos = soup.find_all('div', class_='boxBody')
                    for vuelo in listaVuelos:
                        precio = vuelo.find('span', class_='priceBig').text
                        enlace = html.unescape(vuelo.find('a', class_="operatorName")["href"])
                        empresa = vuelo.find('a', class_="operatorName").getText()
                        enlaceCompra = procesaEnlace(empresa,enlace)
                        if len(enlaceCompra.strip())>0:
                            salida_ida, llegada_ida, aerolinea, salida_vuelta, llegada_vuelta = getDetalle(vuelo)

                            #Añado el vuelo a mi DataFrame
                            dfVuelos = dfVuelos.append({"CAPTURA":datetime.now().strftime("%Y-%m-%d"),'FUENTE':'vuelosbaratos',"ORIGEN":origen, "DESTINO":destino, "SALIDA_IDA":salida_ida, "LLEGADA_IDA": llegada_ida, "SALIDA_VUELTA": salida_vuelta, "LLEGADA_VUELTA": llegada_vuelta, 'URL':enlaceCompra, 'AEROLINEA':aerolinea, 'PRECIO':precio} ,ignore_index=True)

                    #Accedo a la siguiente página
                    navegador.find_element_by_class_name('ChangePageNext').click()
                    sleep(randint(1,4))
                    source = navegador.page_source
                    soup = BeautifulSoup(source).body
                    numErrores = 0
                    error = False
            except Exception as inst:
                print(datetime.now().strftime("%H:%M:%S")," ==>          ERROR al intentar descargar la página. Error número " + str(numErrores))
                print(inst)
                numErrores += 1
                if numErrores > 19:
                    print(datetime.now().strftime("%H:%M:%S")," ==> ERROR SE HA SUPERADO EL NÚMERO MÁXIMO DE ERRORES")
                    break
                sleep(10)
    #Incremento el día
    fecha = fecha + incrementoDia
    fechaVuelta = fecha + incrementoVuelta
    textoFecha = fecha.strftime("%Y-%#m-%#d")
    textoFechaVuelta = fechaVuelta.strftime("%Y-%#m-%#d")
    if path.exists(vuelos_data):
        dfVuelos.to_csv(vuelos_data, mode='a', header=False)
    else:
        dfVuelos.to_csv(vuelos_data)
navegador.quit()
dfVuelos